In [14]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Uyama\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Data Processing

In [5]:
# load the data
import pandas as pd
df = pd.read_csv("twitter_data.csv", encoding="ISO-8859-1")
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [6]:
# assigning names to columns
column_names = ["target", "id","date","flag", "user","text" ]
df = pd.read_csv("twitter_data.csv", names=column_names, encoding = "ISO-8859-1")

In [7]:
df.shape

(1600000, 6)

In [8]:
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [9]:
#check for missing values
df.isnull().sum()c

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [10]:
# checking the distribution of target columns
df["target"].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [11]:
# change the target name from 4 to 1
df.replace({'target': {4: 1}}, inplace=True)

In [12]:
df["target"].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0 ---> negative tweet

1 ---> positive tweet

In [15]:
# applying stemming
import swifter
port_stem = PorterStemmer()

In [16]:
def stemming(content):
    # Remove non-alphabetic characters, keeping spaces
    stemmed_content = re.sub(r'[^a-zA-Z\s]', '', content)
    # Convert to lowercase
    stemmed_content = stemmed_content.lower()
    # Split the content into words
    stemmed_content = stemmed_content.split()
    # Stem each word and remove stopwords
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if word not in stopwords.words("english")]
    # Join the stemmed words back into a string
    stemmed_content = " ".join(stemmed_content)

    return stemmed_content

In [17]:
#create a new column and apply the stemming function
df["stemmed_content"] = df["text"].swifter.apply(stemming)

Pandas Apply:   0%|          | 0/1600000 [00:00<?, ?it/s]

In [18]:
df.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot httptwitpiccomyzl awww that bummer ...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset cant updat facebook text might cri resul...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav im mad cant see


In [19]:
print(df["stemmed_content"])

0          switchfoot httptwitpiccomyzl awww that bummer ...
1          upset cant updat facebook text might cri resul...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                      nationwideclass behav im mad cant see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdbcom cool hear old walt interview httpbli...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [52]:
print(df["target"])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [53]:
# separating the data and label
X = df["stemmed_content"].values
y = df["target"].values

In [51]:
print(X)

['switchfoot httptwitpiccomyzl awww that bummer shoulda got david carr third day'
 'upset cant updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguphh']


In [61]:
print(y)

[0 0 0 ... 1 1 1]


### Split the data into training and testing sets


In [71]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify= y, random_state=42)

In [72]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [73]:
print(X_train)

['paisleypaisley lol get idea far advanc even june yet need third knitter summer group'
 'worst headach ever'
 'ewaniesciuszko sad wont see miss alreadi yeah that perfect come back th'
 ... 'got home meet talk endlessli one coolest guy she ever met smile'
 'bought chocol bar quot win free barquot label didnt win either'
 'misecia said hope dm email sunday']


In [74]:
print(X_test)

['stmdenali ye black red fav color realli want color def look awesom jare'
 'qu buy open hous weekend ampm best valu one bedroom lic long island citi bd httptinyurlcomptnqd'
 'ginoandfran fran greet air that okay hahahaha thank' ...
 'labrat follow also hope atleast get also wish get well soon'
 'feel like havent decent swell sinc last fall hope wave myrtl beach week either least golf'
 'relaxin busi day']


In [75]:
# convert the textual data to numerical values
 # Initialize TF-IDF Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

# Transform the text data into TF-IDF features
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [76]:
print(X_train)

  (0, 191752)	0.2699889384994864
  (0, 528850)	0.2057070428991204
  (0, 313219)	0.43298754546066626
  (0, 551100)	0.2973977531329023
  (0, 393452)	0.16084280709365456
  (0, 615324)	0.20196589015298355
  (0, 297458)	0.24167952967877396
  (0, 153316)	0.18312111187735716
  (0, 5623)	0.31641573446549026
  (0, 159201)	0.22066279226821559
  (0, 262859)	0.22374137247700898
  (0, 180944)	0.12888151740983847
  (0, 335670)	0.1529338066904954
  (0, 419860)	0.4597755974511012
  (1, 153510)	0.5132095392946405
  (1, 201089)	0.5777876782215186
  (1, 604170)	0.6346474357250409
  (2, 543344)	0.27185214392787865
  (2, 38807)	0.1918810401582202
  (2, 97795)	0.20824197554997634
  (2, 426728)	0.31302096230303084
  (2, 544001)	0.21776155551835225
  (2, 613479)	0.2339457202388596
  (2, 15258)	0.2504882393396453
  (2, 370145)	0.19162329852014293
  :	:
  (1279997, 193962)	0.22739640491899754
  (1279997, 509024)	0.2916910245291714
  (1279997, 536219)	0.23918821667051524
  (1279997, 359388)	0.2581513007688756
  

In [77]:
print(X_test)

  (0, 613383)	0.18511180889912876
  (0, 589511)	0.14634802067751218
  (0, 524103)	0.45855844166815857
  (0, 463435)	0.23501558614890705
  (0, 462140)	0.15202629812978538
  (0, 337078)	0.15869499987003843
  (0, 279114)	0.33055066518690546
  (0, 160285)	0.26522549478186236
  (0, 120139)	0.26483393755898565
  (0, 97477)	0.5352955575059065
  (0, 54680)	0.2454966102852299
  (0, 35953)	0.18427253516772754
  (1, 592773)	0.16996763572134646
  (1, 581129)	0.2951072024168845
  (1, 445638)	0.33101694554235694
  (1, 414674)	0.20697359525719528
  (1, 412676)	0.13725322888309424
  (1, 336751)	0.17360105510600915
  (1, 328653)	0.40100695182582297
  (1, 272293)	0.25535475907252503
  (1, 213320)	0.18205770933359394
  (1, 92033)	0.21973903048938576
  (1, 71634)	0.19696833977298697
  (1, 49530)	0.17470114715875446
  (1, 46165)	0.28281841880029335
  :	:
  (319997, 513369)	0.27562451740910365
  (319997, 211924)	0.22965160814847113
  (319997, 180944)	0.3702493789986563
  (319997, 167855)	0.25765629121838257

### Model Training

In [78]:
model = LogisticRegression(max_iter=1000)

model.fit(X_train,y_train)

LogisticRegression(max_iter=1000)

### Model Evaluation

In [79]:
from sklearn.metrics import accuracy_score

# Predict on the training data
y_train_pred = model.predict(X_train)

# Calculate the accuracy on the training data
model_accuracy_train = accuracy_score(y_train, y_train_pred)

# Print the accuracy score
print("Training Accuracy:", model_accuracy_train)


Training Accuracy: 0.80604453125


In [81]:
print("X_train shape:", X_train.shape)
print("y_train shape:", len(y_train))
print("X_test shape:", X_test.shape)
print("y_test shape:", len(y_test))


X_train shape: (1280000, 622188)
y_train shape: 1280000
X_test shape: (320000, 622188)
y_test shape: 320000


In [82]:
# Fit the model on the training data
model.fit(X_train, y_train)

# Predict on the test data
y_test_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", accuracy)


Test Accuracy: 0.780865625


In [83]:
#save the trained model
import pickle
pickle.dump(model,open("twitter_model.pkl","wb"))

### Making predictions using the saved model

In [86]:
# Load the saved model
with open("twitter_model.pkl", 'rb') as file:
    loaded_model = pickle.load(file)

In [90]:
# Extract the 200th sample from X_test
X_new = X_test[200]  # Use indexing to get the 200th sample

# Check the true label for the 200th sample
print(y_test[200])

0


In [91]:
prediction = loaded_model.predict(X_new)

# Print the prediction
print("Prediction:", prediction)

# Check if the prediction is positive or negative
if prediction[0] == 0:
    print("Negative Tweet")
else:
    print("Positive Tweet")

Prediction: [0]
Negative Tweet


In [92]:
X_new = X_test[68] 
print(y_test[68])

prediction = loaded_model.predict(X_new)

# Print the prediction
print("Prediction:", prediction)

# Check if the prediction is positive or negative
if prediction[0] == 0:
    print("Negative Tweet")
else:
    print("Positive Tweet")

1
Prediction: [1]
Positive Tweet
